# Check if the keypoints are duplicated

\\wsl.localhost\Ubuntu\home\pkhamchuai\codes\spppt-2\output\SIFT\SIFT_20100_0.0001_0_1_1_20240705-150230_BFMatcher_RANSAC_test

In [7]:
import argparse
from run_SP import *
from utils.utils1 import draw_lines

In [8]:
def run(model_params, method='LMEDS', plot=1):
    # Initialize SuperPointFrontend
    superpoint = SuperPointFrontend('utils/superpoint_v1.pth', nms_dist=4,
                          conf_thresh=0.015,
                          nn_thresh=0.7, cuda=True)
    
    test_dataset = datagen(model_params.dataset, False, model_params.sup)

    # Create output directory
    output_dir = f"output/{args.model}_{model_params.get_model_code()}_{timestamp}_{method}_test"
    os.makedirs(output_dir, exist_ok=True)

    metrics = []
    # create a csv file to store the metrics
    csv_file = f"{output_dir}/metrics.csv"

    testbar = tqdm(test_dataset, desc=f'Testing:')
    for i, data in enumerate(testbar, 0):
        if i != 90:
            continue
        if i == 90:
            # Get images and affine parameters
            source_image, target_image, affine_params_true, points1, points2, points1_2_true = data
                
            # process images
            source_image = process_image(source_image)
            target_image = process_image(target_image)

            # need to perform superpoint again because descriptors are not saved
            # Process the first image
            points1, desc1, heatmap1 = superpoint(source_image)
            # Process the second image
            points2, desc2, heatmap2 = superpoint(target_image)

            # match the points between the two images
            tracker = PointTracker(5, nn_thresh=0.7)
            matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=0.7)

            # take the elements from points1 and points2 using the matches as indices
            matches1 = points1[:2, matches[0, :].astype(int)]
            matches2 = points2[:2, matches[1, :].astype(int)]

            # create affine transform matrix from points1 to points2
            # and apply it to points1
            try:
                if method == 'RANSAC':
                    affine_transform1 = cv2.estimateAffinePartial2D(matches1.T, matches2.T, method=cv2.RANSAC)
                elif method == 'LMEDS':
                    affine_transform1 = cv2.estimateAffinePartial2D(matches1.T, matches2.T, method=cv2.LMEDS)
                matches1_transformed = cv2.transform(matches1.T[None, :, :], affine_transform1[0])
                matches1_transformed = matches1_transformed[0].T
                # transform image 1 and 2 using the affine transform matrix
                transformed_source_affine = cv2.warpAffine(source_image, affine_transform1[0], (256, 256))
            except cv2.error:
                print(f"Error: {i}")
                # set affine_transform1 to identity affine matrix
                affine_transform1 = np.array([[[1, 0, 0], [0, 1, 0]]])
                matches1_transformed = matches1
                transformed_source_affine = source_image
            
            # mse12 = np.mean((matches1_transformed - matches2)**2)
            # tre12 = np.mean(np.sqrt(np.sum((matches1_transformed - matches2)**2, axis=0)))

            if i < 100 and plot == 1:
                plot_ = 1
            elif i < 100 and plot == 2:
                plot_ = 2
            else:
                plot_ = 0

            results = DL_affine_plot(f"test", output_dir,
                    f"{i}", "SP", source_image, target_image, \
                    transformed_source_affine, \
                    matches1, matches2, matches1_transformed, desc1, desc2, 
                    affine_params_true=affine_params_true,
                    affine_params_predict=affine_transform1[0], 
                    heatmap1=heatmap1, heatmap2=heatmap2, plot=plot_)


            # calculate metrics
            # matches1_transformed = results[0]
            mse_before = results[1]
            mse12 = results[2]
            tre_before = results[3]
            tre12 = results[4]
            mse12_image_before = results[5]
            mse12_image = results[6]
            ssim12_image_before = results[7]
            ssim12_image = results[8]

    return source_image, target_image, transformed_source_affine, matches1, matches2, matches1_transformed
            





In [9]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Deep Learning for Image Registration')    
    parser.add_argument('--dataset', type=int, default=2, help='dataset number')
    parser.add_argument('--sup', type=int, default=1, help='supervised learning (1) or unsupervised learning (0)')
    parser.add_argument('--image', type=int, default=1, help='image used for training')
    parser.add_argument('--heatmaps', type=int, default=0, help='use heatmaps (1) or not (0)')
    parser.add_argument('--loss_image', type=int, default=0, help='loss function for image registration')
    parser.add_argument('--num_epochs', type=int, default=1, help='number of epochs')
    parser.add_argument('--learning_rate', type=float, default=1e-4, help='learning rate')
    parser.add_argument('--decay_rate', type=float, default=0.96, help='decay rate')
    parser.add_argument('--model', type=str, default='SP', help='which model to use')
    parser.add_argument('--model_path', type=str, default=None, help='path to model to load')
    parser.add_argument('--plot', type=int, default=1, help='plot the results')
    parser.add_argument('--method', type=str, default='LMEDS', help='method to use for affine transformation')
    args = parser.parse_args()

    model_params = ModelParams(dataset=args.dataset, sup=args.sup, image=args.image, #heatmaps=args.heatmaps, 
                                loss_image=args.loss_image, num_epochs=args.num_epochs, 
                                learning_rate=args.learning_rate, decay_rate=args.decay_rate)
    model_params.print_explanation()

    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

    # run the function
    source_image, target_image, transformed_source_affine, matches1, matches2, matches1_transformed = run(model_params, method='RANSAC', plot=1)

    # plot the images
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(source_image, cmap='gray')
    ax[0].scatter(matches1[0], matches1[1], c='r', s=5)
    ax[0].set_title('Source Image')
    ax[1].imshow(target_image, cmap='gray')
    ax[1].scatter(matches2[0], matches2[1], c='r', s=5)
    ax[1].set_title('Target Image')
    ax[2].imshow(transformed_source_affine, cmap='gray')
    ax[2].scatter(matches1_transformed[0], matches1_transformed[1], c='r', s=5)
    ax[2].set_title('Transformed Source Image')
    plt.show()

usage: ipykernel_launcher.py [-h] [--dataset DATASET] [--sup SUP]
                             [--image IMAGE] [--heatmaps HEATMAPS]
                             [--loss_image LOSS_IMAGE]
                             [--num_epochs NUM_EPOCHS]
                             [--learning_rate LEARNING_RATE]
                             [--decay_rate DECAY_RATE] [--model MODEL]
                             [--model_path MODEL_PATH] [--plot PLOT]
                             [--method METHOD]
ipykernel_launcher.py: error: unrecognized arguments: --f=/home/pkhamchuai/.local/share/jupyter/runtime/kernel-v2-3264JdBR7U1JI62t.json


SystemExit: 2

/home/pkhamchuai/codes/spppt-2/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
